In [3]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [5]:
zip_path = r'C:\data\PAKDD2010.zip'
extract_folder = r'C:\data\PAKDD2010_extracted'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Файлы распакованы в:", extract_folder)

Файлы распакованы в: C:\data\PAKDD2010_extracted


In [7]:
import zipfile

zip_path = r'C:\data\PAKDD2010.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    print("Содержимое ZIP-файла:")
    zip_ref.printdir()

Содержимое ZIP-файла:
File Name                                             Modified             Size
PAKDD 2010/                                    2025-06-10 18:06:06            0
PAKDD 2010/PAKDD-2010 training data.zip        2025-06-10 18:05:12      2106886
PAKDD 2010/PAKDD2010_VariablesList.XLS         2025-06-10 18:05:12        25600
PAKDD 2010/Prediction_Data.zip                 2025-06-10 18:05:12       833476


In [9]:
inner_zip_path = 'PAKDD 2010/PAKDD-2010 training data.zip'

with zipfile.ZipFile(zip_path, 'r') as outer_zip:
    with outer_zip.open(inner_zip_path) as inner_zip_file:
        inner_zip = zipfile.ZipFile(inner_zip_file)
        print("Содержимое внутреннего ZIP:")
        inner_zip.printdir()

Содержимое внутреннего ZIP:
File Name                                             Modified             Size
PAKDD2010_Modeling_Data.txt                    2010-03-17 16:33:16      8394946


In [11]:
with zipfile.ZipFile(zip_path, 'r') as outer_zip:
    with outer_zip.open(inner_zip_path) as inner_zip_file:
        inner_zip = zipfile.ZipFile(inner_zip_file)

        # Открываем файл как байты и указываем кодировку при чтении
        with inner_zip.open('PAKDD2010_Modeling_Data.txt') as file:
            df_train = pd.read_csv(file, sep='\t', encoding='latin1')
            print(df_train.shape)
            print(df_train.head())

(49999, 54)
   1  C   5    Web  0  1.1  F  6  1.2  0.1  ... 0.11 0.12  0.13 0.14 1.8 N.3  \
0  2  C  15  Carga  0    1  F  2    0    0  ...    0    0     0    0   1   N   
1  3  C   5    Web  0    1  F  2    0    0  ...    0    0     0    0   1   N   
2  4  C  20    Web  0    1  F  2    0    0  ...    0    0     0    0   1   N   
3  5  C  10    Web  0    1  M  2    0    0  ...    0    0     0    0   1   N   
4  6  C  10      0  0    1  M  2    0    0  ...    0    0     0    0   2   N   

   32  595  595.1  1.9  
0  34  230    230    1  
1  27  591    591    0  
2  61  545    545    0  
3  48  235    235    1  
4  40  371    371    1  

[5 rows x 54 columns]


C:\Users\Admin\AppData\Local\Temp\ipykernel_10752\2524740912.py:7: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(file, sep='\t', encoding='latin1')


In [13]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [15]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    with zip_ref.open('PAKDD 2010/PAKDD2010_VariablesList.XLS') as file:
        df_vars = pd.read_excel(file)
        print(df_vars.head())

   Var_Id                    Var_Title  \
0       1                    ID_CLIENT   
1       2                   CLERK_TYPE   
2       3                  PAYMENT_DAY   
3       4  APPLICATION_SUBMISSION_TYPE   
4       5       QUANT_ADDITIONAL_CARDS   

                                     Var_Description  \
0  Sequential number for the applicant (to be use...   
1                                       Not informed   
2  Day of the month for bill payment, chosen by t...   
3  Indicates if the application was submitted via...   
4  Quantity of additional cards asked for in the ...   

                       Field_Content  
0  1-50000, 50001-70000, 70001-90000  
1                                  C  
2                    1,5,10,15,20,25  
3                         Web, Carga  
4                           1,2,NULL  


In [17]:
# Создаем словарь {номер_столбца: Var_Title}
column_mapping = dict(zip(df_vars['Var_Id'], df_vars['Var_Title']))

# Применяем к датафрейму
df_train.columns = [column_mapping.get(i+1, f"col_{i+1}") for i in range(df_train.shape[1])]

# Проверяем
print(df_train.head())

   ID_CLIENT CLERK_TYPE  PAYMENT_DAY APPLICATION_SUBMISSION_TYPE  \
0          2          C           15                       Carga   
1          3          C            5                         Web   
2          4          C           20                         Web   
3          5          C           10                         Web   
4          6          C           10                           0   

   QUANT_ADDITIONAL_CARDS  POSTAL_ADDRESS_TYPE SEX  MARITAL_STATUS  \
0                       0                    1   F               2   
1                       0                    1   F               2   
2                       0                    1   F               2   
3                       0                    1   M               2   
4                       0                    1   M               2   

   QUANT_DEPENDANTS  EDUCATION_LEVEL  ... FLAG_HOME_ADDRESS_DOCUMENT FLAG_RG  \
0                 0                0  ...                          0       0   
1         

In [19]:
print(df_vars[df_vars['Var_Description'].str.contains('default|delinquency|target', case=False, na=False)])

    Var_Id           Var_Title  \
49      50    FLAG_ACSP_RECORD   
53      54  TARGET_LABEL_BAD=1   

                                      Var_Description Field_Content  
49  Flag indicating if the applicant has any previ...          Y, N  
53                     Target Variable: BAD=1, GOOD=0           NaN  


In [21]:
target_col = 'TARGET_LABEL_BAD=1'

# Уникальные значения
print("Уникальные значения в целевой переменной:")
print(df_train[target_col].unique())

# Пропуски
print("\nКоличество пропущенных значений в целевой переменной:")
print(df_train[target_col].isna().sum())

Уникальные значения в целевой переменной:
[1 0]

Количество пропущенных значений в целевой переменной:
0


In [23]:
print("\nТип данных целевой переменной:", df_train[target_col].dtype)


Тип данных целевой переменной: int64


In [25]:
# Удалим ID_CLIENT, если он не нужен для модели
if 'ID_CLIENT' in df_train.columns:
    df_train.drop('ID_CLIENT', axis=1, inplace=True)

In [27]:
target_col = 'TARGET_LABEL_BAD=1'

X = df_train.drop(columns=[target_col])
y = df_train[target_col]

In [29]:
categorical_cols = df_vars[df_vars['Field_Content'].notna() &
                           df_vars['Field_Content'].str.contains(',', na=False)]['Var_Title'].tolist()

# Убедимся, что целевая переменная не попадает в список
categorical_cols = [col for col in categorical_cols if col != target_col]

print("Категориальные признаки:", categorical_cols)

Категориальные признаки: ['ID_CLIENT', 'PAYMENT_DAY', 'APPLICATION_SUBMISSION_TYPE', 'QUANT_ADDITIONAL_CARDS', 'SEX', 'MARITAL_STATUS', 'QUANT_DEPENDANTS', 'EDUCATION_LEVEL', 'STATE_OF_BIRTH', 'NACIONALITY', 'FLAG_RESIDENCIAL_PHONE', 'RESIDENCE_TYPE', 'MONTHS_IN_RESIDENCE', 'FLAG_MOBILE_PHONE', 'FLAG_OTHER_CARDS', 'QUANT_BANKING_ACCOUNTS', 'QUANT_SPECIAL_BANKING_ACCOUNTS', 'COMPANY', 'FLAG_PROFESSIONAL_PHONE', 'PROFESSION_CODE', 'OCCUPATION_TYPE', 'MATE_PROFESSION_CODE', 'EDUCATION_LEVEL', 'PRODUCT', 'FLAG_ACSP_RECORD']


In [31]:
categorical_cols = df_vars[df_vars['Field_Content'].notna() &
                           df_vars['Field_Content'].str.contains(',', na=False)]['Var_Title'].tolist()

# Убедимся, что целевая переменная и несуществующие столбцы не попадают в список
categorical_cols = [col for col in categorical_cols if col in X.columns and col != target_col]

print("Категориальные признаки:", categorical_cols)

Категориальные признаки: ['PAYMENT_DAY', 'APPLICATION_SUBMISSION_TYPE', 'QUANT_ADDITIONAL_CARDS', 'SEX', 'MARITAL_STATUS', 'QUANT_DEPENDANTS', 'EDUCATION_LEVEL', 'STATE_OF_BIRTH', 'NACIONALITY', 'FLAG_RESIDENCIAL_PHONE', 'RESIDENCE_TYPE', 'MONTHS_IN_RESIDENCE', 'FLAG_MOBILE_PHONE', 'FLAG_OTHER_CARDS', 'QUANT_BANKING_ACCOUNTS', 'QUANT_SPECIAL_BANKING_ACCOUNTS', 'COMPANY', 'FLAG_PROFESSIONAL_PHONE', 'PROFESSION_CODE', 'OCCUPATION_TYPE', 'MATE_PROFESSION_CODE', 'EDUCATION_LEVEL', 'PRODUCT', 'FLAG_ACSP_RECORD']


In [33]:
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

In [35]:
categorical_cols = df_vars[df_vars['Field_Content'].notna() &
                           df_vars['Field_Content'].str.contains(',', na=False)]['Var_Title'].tolist()

# Убедимся, что целевая переменная и несуществующие столбцы не попадают в список
categorical_cols = [col for col in categorical_cols if col in X.columns and col != target_col]

print("Категориальные признаки:", categorical_cols)

Категориальные признаки: []


In [37]:
import os

print("Текущая директория:", os.getcwd())
print("Файлы в текущей директории:")
print(os.listdir())

Текущая директория: C:\Users\Admin
Файлы в текущей директории:
['.anaconda', '.conda', '.condarc', '.continuum', '.docker', '.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '.spyder-py3', '3D Objects', 'anaconda3', 'anaconda_projects', 'AppData', 'Application Data', 'catboost_credit_scoring_model.cbm', 'catboost_info', 'Contacts', 'Cookies', 'data', 'Desktop', 'Documents', 'Downloads', 'Favorites', 'IntelGraphicsProfiles', 'Jedi', 'Links', 'Local Settings', 'Music', 'mu_code', 'NetHood', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{9852813a-5d6c-11ec-9ed0-080027d53198}.TM.blf', 'NTUSER.DAT{9852813a-5d6c-11ec-9ed0-080027d53198}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{9852813a-5d6c-11ec-9ed0-080027d53198}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'Pictures', 'PrintHood', 'Python-Copy1.txt', 'Python-Copy2.txt', 'Python.txt', 'Recent', 'Saved Games', 'Searches', 'SendTo', 'submission_final.csv', 'Untitled Восемьдесят первый